Jupyter notebook for Capstone project

Import the libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests

Retrieve the data from the wikipidea page

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text

Create a BeautifulSoup object

In [3]:
soup=BeautifulSoup(data,'html5lib')

Following the hints, assign the corresponding cells to a list of dictionaries. The list of dictionaries is converted to dataframe

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Find the shape of the dataframe

In [6]:
df.shape

(103, 3)

In [14]:
import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None
postal_code=df.iloc[2,0]

In [19]:
# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('M5A, Toronto, Ontario')#.format(postal_code))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
latitude

KeyboardInterrupt: 

Unable to extract data from geocoder

In [39]:
latlng=pd.read_csv('Geospatial_Coordinates.csv')
latlng.head(20)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [40]:
latitude=[]
longitude=[]
for postal_code in df['PostalCode']:
    latitude.append(latlng[latlng['Postal Code']==postal_code]['Latitude'])
    longitude.append(latlng[latlng['Postal Code']==postal_code]['Longitude'])
df[['Latitude']]=latitude
df[['Longitude']]=longitude

In [41]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Queen's Park,Ontario Provincial Government,43.6623,-79.3895
